# Predicting credit card debt from UCI dataset

## by Christopher Forsythe, Daniel Deleon, Jose Garcia

### Introduction

### Exploration

### Preprocessing

### Experiments/Dimensionality Reduction

### Ensemble Learning

### SVM

### Gridsearch

### Analyses

### Performance